In [10]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding , Dense, Dropout,Conv1D,GlobalMaxPool1D, concatenate

# 데이터 읽어오기 1
train_file = './chatbot_data.csv'
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀸스 벡터 2
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.oov_token = None
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

MAX_SEQ_LEN = 15 # 단어 시퀸스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen = MAX_SEQ_LEN,padding='post')

#학습용, 검증용, 테스트용 데이터셋 생성 3
#학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.take(train_size + val_size).take(test_size).batch(20)

#하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 #전체 단어수

#CNN 모델 정의 4
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)


# 3,4,5-gram 이후 합치기
concat = concatenate([pool1,pool2,pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(128, activation=tf.nn.relu)(logits)

# 모델 생성 5
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 학습 6 
model.fit(train_ds, validation_data = val_ds, epochs=EPOCH, verbose = 1)

# 모델 평가(테스트 데이터셋 이용) 7 
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss : %f' % (loss))

# 모델 저장 8
model.save('cnn_model.h5')


Epoch 1/5
414/414 [==============================] - 9s 19ms/step - loss: 1.3204 - accuracy: 0.4805 - val_loss: 0.8543 - val_accuracy: 0.5825
Epoch 2/5
414/414 [==============================] - 7s 18ms/step - loss: 0.8939 - accuracy: 0.6046 - val_loss: 0.7197 - val_accuracy: 0.6523
Epoch 3/5
414/414 [==============================] - 8s 18ms/step - loss: 0.8490 - accuracy: 0.6387 - val_loss: 0.6620 - val_accuracy: 0.6595
Epoch 4/5
414/414 [==============================] - 7s 18ms/step - loss: 0.7158 - accuracy: 0.7540 - val_loss: 0.5036 - val_accuracy: 0.8363
Epoch 5/5
60/60 [==============================] - 0s 2ms/step - loss: 0.3591 - accuracy: 0.9399
Accuracy: 93.993235
loss : 0.359129
